In [3]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import geopandas as gpd
# import geemap


In [7]:
# Load Europe shapefile into a GeoDataFrame
Europe_path = "./data/raw/europe/Europe.shp"
Europe = gpd.read_file(Europe_path)
Europe.crs


DataSourceError: ./data/raw/europe/Europe.shp: No such file or directory

In [41]:
Europe

,NAME,ORGN_NAME,geometry
0,Albania,Shqipëria,"MULTIPOLYGON (((19.50115 40.9623, 19.50563 40...."
1,Andorra,Andorra,"POLYGON ((1.43992 42.60649, 1.45041 42.60596, ..."
2,Austria,Österreich,"POLYGON ((16 48.77775, 16 48.78252, 16.00811 4..."
3,Belgium,België / Belgique,"POLYGON ((5 49.79374, 4.99724 49.79696, 4.9900..."
4,Bosnia Herzegovina,Bosna i Hercegovina,"POLYGON ((19.22947 43.53458, 19.22925 43.53597..."
5,Croatia,Hrvatska,"MULTIPOLYGON (((14.30038 44.50156, 14.28972 44..."
6,Czech Republic,Cesko,"POLYGON ((14.82523 50.87399, 14.83687 50.86996..."
7,Denmark,Danmark,"MULTIPOLYGON (((11.99978 54.94118, 11.98534 54..."
8,Estonia,Eesti,"MULTIPOLYGON (((23.97511 58.09691, 23.96645 58..."
9,Finland,Suomi,"MULTIPOLYGON (((22.0731 60.2283, 22.06502 60.2..."


In [49]:
nonEurope_country = ['Turkey', 'Kazakhstan', 'Armenia', 'Azerbaijan', 'Georgia','Russia','Svalbard (Norway)','Jan Mayen (Norway)'] #remove non-European countries (EEA38)
Europe_filtered = Europe[~Europe['NAME'].isin(nonEurope_country)]
Europe_filtered = Europe_filtered[['NAME','geometry']]
Europe_outline = Europe_filtered.dissolve() #Get outline of Europe for clipping
Europe_outline.loc[:, 'NAME'] = 'Europe'


In [56]:
# Load effis shapefile into a GeoDataFrame
effis_path = "data/raw/effis_layer/modis.ba.poly.shp"
effis = gpd.read_file(effis_path)
effis = effis.to_crs(Europe.crs)

# Create a new column based on the conditions
effis['ForestType'] = 'Non-Forest'  # Default value
effis.loc[effis['BROADLEA'].astype(float) > 50, 'ForestType'] = 'Broadleaf'
effis.loc[effis['CONIFER'].astype(float) > 50, 'ForestType'] = 'Conifer'
effis.loc[effis['MIXED'].astype(float) > 50, 'ForestType'] = 'Mixed'
# Remove non-forested rows
effis_forests = effis[effis['ForestType'] != 'Non-Forest']
# Fix date
effis_forests.loc[:,'Date'] = pd.to_datetime(effis_forests['FIREDATE'], errors='coerce')
effis_forests['Date'] = effis_forests['Date'].dt.strftime('%Y-%m-%d')
# Select subset of variables
effis_forests = effis_forests[['Date', 'COUNTRY', 'ForestType', 'AREA_HA', 'geometry']]


In [58]:
effis_forests

,Date,COUNTRY,ForestType,AREA_HA,geometry
14,2000-01-01,IT,Mixed,70,"POLYGON ((16.9034 39.39853, 16.90282 39.3937, ..."
31,2000-01-01,FR,Broadleaf,47,"POLYGON ((9.28158 42.5094, 9.27938 42.50779, 9..."
76,2000-01-01,ES,Broadleaf,53,"POLYGON ((-6.48548 42.72917, -6.48424 42.72443..."
77,2000-01-01,IT,Broadleaf,116,"POLYGON ((16.8291 39.40912, 16.82796 39.39944,..."
81,2000-01-01,IT,Conifer,73,"POLYGON ((16.14378 41.84931, 16.13063 41.84858..."
...,...,...,...,...,...
87812,2024-11-30,LY,Mixed,3,"POLYGON ((15.15892 32.01566, 15.15892 32.0156,..."
87832,2024-09-25,UA,Mixed,12,"POLYGON ((34.20177 51.3803, 34.20188 51.37999,..."
87839,2024-09-25,UA,Mixed,98,"POLYGON ((32.71688 52.21854, 32.71648 52.21853..."
87840,2024-09-25,UA,Mixed,58,"POLYGON ((32.76401 52.22566, 32.76352 52.2256,..."


In [59]:
effis_clipped = gpd.clip(effis_forests, Europe_outline)


In [60]:
effis_clipped

,Date,COUNTRY,ForestType,AREA_HA,geometry
80109,2024-08-21,EL,Conifer,3,"POLYGON ((27.74844 36.14392, 27.74847 36.1434,..."
34387,2020-07-16,EL,Mixed,59,"POLYGON ((24.02732 37.69824, 24.02807 37.6975,..."
712,2000-01-01,EL,Mixed,846,"POLYGON ((24.01099 37.72039, 24.013 37.72009, ..."
75043,2024-06-29,EL,Mixed,57,"POLYGON ((23.79226 38.18259, 23.79176 38.18186..."
27465,2018-10-25,EL,Mixed,610,"POLYGON ((23.94573 40.10388, 23.94572 40.10391..."
...,...,...,...,...,...
26774,2018-05-31,SE,Conifer,58,"POLYGON ((17.92254 64.32946, 17.92275 64.32945..."
62402,2023-06-17,SE,Conifer,25,"POLYGON ((17.6619 64.47057, 17.66232 64.47035,..."
74805,2024-05-30,SE,Conifer,17,"POLYGON ((18.18329 66.05112, 18.18387 66.05074..."
41923,2021-05-30,SE,Conifer,1,"POLYGON ((16.53693 63.50015, 16.53727 63.50019..."


In [61]:
m = geemap.Map()

# Add the GeoDataFrame to the map
m.add_gdf(effis_clipped, layer_name="EFFIS Forest Fires")
m.add_gdf(Europe_outline, layer_name="Europe")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [62]:
# Save
effis_clipped.to_file("data/processed/Fire_Data/EFFIS/effis-filtered.shp", driver="ESRI Shapefile")